In [3]:
import sys
import numpy as np
import pandas as pd
sys.path.append("../code/")

In [4]:
from dataloader import APPLIANCE_ORDER, get_train_test
from ddsc import SparseCoding, reshape_for_sc

In [5]:
# tensor = np.load('../2015-5appliances-true-agg.npy')
ag_tensor = np.load('../2015-5appliances.numpy.npy')
true_tensor = np.load("../2015-5appliances-true-agg.npy")
from sklearn.metrics import mean_absolute_error



In [10]:
ag_agg = ag_tensor[:, 0, :, :]
true_agg = true_tensor[:, 0, :, :]
residual_agg = true_agg - ag_agg

In [12]:
residual_agg = residual_agg.reshape(68, -1)

In [14]:
residual_agg.shape

(68, 2688)

In [15]:
from sklearn.cluster import KMeans

In [53]:
kmeans = KMeans(n_clusters=3, random_state=0).fit(residual_agg)
kmeans.labels_

array([1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 2,
       0, 1, 1, 0, 1, 0, 2, 1, 1, 1, 1, 0, 1, 1, 0, 1, 2, 1, 1, 1, 1, 0, 2,
       2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 0, 1], dtype=int32)

In [54]:
cluster = {}
for i in range(3):
    cluster[i] = [x for x in range(68) if kmeans.labels_[x] == i]

In [63]:
np.save("../2015-5appliances-true-agg-c0.npy", true_agg[cluster[0]])
np.save("../2015-5appliances-true-agg-c1.npy", true_agg[cluster[1]])
np.save("../2015-5appliances-true-agg-c2.npy", true_agg[cluster[2]])

In [56]:
def mean_predictions(tensor):
    out = []
    for cur_fold in range(5):
        train, test = get_train_test_tensor(tensor, num_folds=num_folds, fold_num=cur_fold)
        pred = np.zeros_like(test[:, 1:, :, :])
        print(pred.shape)
        
        for appliance in range(test.shape[1]-1):
            for day in range(test.shape[2]):
                pred[:, appliance, day, :] = train[:, appliance+1, day, :].mean(axis=0)
        out.append(pred)
    return np.concatenate(out)

In [58]:
from sklearn.model_selection import KFold
error_mean = {}
num_folds = 5
for i in range(3):
    error_mean[i] = {}
    gt = true_tensor[cluster[i], 1:, :, :]
    pred = mean_predictions(true_tensor[cluster[i]])
    pred = np.minimum(pred, true_tensor[cluster[i], 0:1, :,:])

    error_mean[i] = {APPLIANCE_ORDER[i+1]:mean_absolute_error(pred[:, i,:,:].flatten(), 
                                                                       gt[:, i, :, :].flatten()) for i in range(pred.shape[1])}

(3, 5, 112, 24)
(3, 5, 112, 24)
(2, 5, 112, 24)
(2, 5, 112, 24)
(2, 5, 112, 24)
(10, 5, 112, 24)
(10, 5, 112, 24)
(10, 5, 112, 24)
(9, 5, 112, 24)
(9, 5, 112, 24)
(2, 5, 112, 24)
(2, 5, 112, 24)
(2, 5, 112, 24)
(1, 5, 112, 24)
(1, 5, 112, 24)


In [62]:
pd.DataFrame(error_mean).mean(axis=1)

dr        138.987168
dw         35.703705
fridge     31.557518
hvac      591.487861
mw         16.035679
dtype: float64

In [ ]:
dataset = 2
err_mean ={}
gt = tensor[:, 1:, :, :]
pred = mean_predictions(dataset)
# Clamping prediction to aggregate
pred = np.minimum(pred, tensor[:, 0:1, :,:])
err_mean = {APPLIANCE_ORDER[i+1]:mean_absolute_error(pred[:, i,:,:].flatten(), 
                                                                       gt[:, i, :, :].flatten()) for i in range(pred.shape[1])}